In [1]:
import pandas as pd
import json
import preprocessor as pp

In [2]:
# merge suggestions from :
#- datafiller
#- original list
#- queries found in logs

In [3]:
# we load and apply standard weight

datafiller = pd.read_csv("datafiller_suggestions.csv", names=['idx', 'suggestion'])
datafiller["weight"] = 10

In [4]:
crawled = pd.read_csv("crawled_suggestions.csv", names=['idx', 'suggestion'])
crawled["weight"] = 1

In [5]:
popular_raw = pd.read_excel("../log-analysis/most_popular_queries.xlsx")
popular = pd.DataFrame(popular_raw['unique_values']).rename(columns={'unique_values': 'suggestion'})
popular["weight"] = 2


In [6]:
# then we apply the autocorrection
def autocorrect(entities):
    entities.suggestion = pp.replace_standard_issues(entities.suggestion)
    return entities[entities.suggestion.apply(lambda e : pp.suggestion_filter(e))].reset_index(drop=True)

In [7]:
datafiller_corrected = autocorrect(datafiller)

Error : prudhomme
Error : oeuvres
Error : indeterminee
Error : surcroit
Error : conge
Error : intragroupe
Error : ue
Error : nao
Error : cesu
Error : tesa
Error : tese
Error : diplomante
Error : ijss
Error : ijss
Error : interdicton
Error : interimaire
Error : definition
Error : ferié
Error : journee
Error : feriés
Error : khimâr
Error : l1224
Error : toussaint
Error : laicité
Error : cheque
Error : video
Error : injustifé
Error : demenagement
Error : assedic
Error : defendre
Error : temoignage
Error : declaration
Error : space
Error : dej
Error : pei
Error : pei
Error : plateformes
Error : cesu
Error : alsace
Error : vrp
Error : oeuvre
Error : securite
Error : prud
Error : prudhommes
Error : 7h
Error : invalidite
Error : -elle
Error : quote
Error : recuperation
Error : recuperer
Error : securite
Error : legiFrance
Error : reglementation
Error : velib
Error : pass
Error : conventionelle
Error : sénior
Error : ctt
Error : prud’hommal
Error : apres
Error : soufFrance
Error : temperature


In [8]:
popular_corrected = autocorrect(popular)

Error : démissioner
Error : congès
Error : évènement
Error : convzention
Error : évènements
Error : bdes
Error : greve
Error : évenements
Error : evenement
Error : etrangers
Error : conventimajoration
Error : nao
Error : etam
Error : réglement
Error : r4614
Error : legal
Error : supplementaires
Error : epi
Error : l3111
Error : reglementation
Error : remuneration
Error : duree
Error : rss
Error : l3121
Error : batiment
Error : appointement
Error : l1242
Error : pse
Error : atex
Error : regle
Error : pass
Error : suppélementaires
Error : ossibilité
Error : etam
Error : cssct
Error : caces
Error : regles
Error : paerosnne
Error : ald
Error : epa
Error : coactivité
Error : etat
Error : cssct
Error : licenciemnt
Error : cesu


In [9]:
crawled_corrected = autocorrect(crawled)

Error : value


In [10]:
# finally we merge and disambiguate
all_suggestions = pd.concat([popular_corrected, crawled_corrected, datafiller_corrected], ignore_index=True).drop(['idx'], axis=1)


/cdtn/ubuntu-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [10]:
all_suggestions.shape

(3029, 2)

,suggestion,weight
0,renouvellement période d'essai,2
1,rupture conventionnelle,2
2,licenciement,2
3,congés familiaux,2
4,durée du préavis dans la boulangerie,2
...,...,...
3024,inaptitude au travail et chômage,10
3025,indemnisation accident du travail,10
3026,indemnisation chômage partiel,10
3027,qui verse les indemnités journalières ?,10


In [28]:
from datasketch import MinHash, MinHashLSH
from nltk import ngrams
import re

def prepro_hash(sugg):
    no_punct = re.sub(r'[^\w\s]', ' ', sugg)
    return " ".join([t.text.strip() for t in pp.to_tokens(no_punct)])

def disambiguate_entities(entities):
    
    lsh = MinHashLSH(threshold=0.85, num_perm=128)

    minhashes = {}
    for c, i in enumerate(entities.suggestion):
        minhash = MinHash(num_perm=128)
        pi = prepro_hash(i)
        for d in ngrams(pi, 3):
            minhash.update("".join(d).encode('utf-8'))
        lsh.insert(c, minhash)
        minhashes[c] = minhash
    
    sets = []


    def search_candidate(doc):
        minhash = MinHash(num_perm=128)
        pi = prepro_hash(doc)
        for d in ngrams(pi, 3):
            minhash.update("".join(d).encode('utf-8'))

        search = lsh.query(minhash)

        if (len(search) > 1):
            print("Disambiguation match : " + doc)
            print([entities.iloc[i].suggestion for i in search])
            print()
            sets.append(search)
            return False
        else:
            return True
    
    unambiguous = entities[entities.suggestion.apply(lambda e : search_candidate(e))]
    
    acc = []

    for s in sets:
        se = set(s)
        match = None
        for ses in acc:
            if len(se.intersection(ses)) > 0:
                match = ses
                break
        if (match != None):
            acc.remove(match)
            acc.append(match&se)
        else :
            acc.append(se)
            
    # choose between ambiguous the one with bigger weight (most reliable)
    disambiguated = []
    for a in acc :        
        w = 0
        choice = None
        for i in list(a):
            e = entities.iloc[i]
            if (e.weight > w):
                choice = e
                
        # boost the weight, as found in several
        choice['weight'] = choice["weight"] + len(a)
        disambiguated.append(choice)
        
    return pd.concat([unambiguous, pd.DataFrame(disambiguated)])

In [30]:
disambiguated = disambiguate_entities(all_suggestions)

Disambiguation match : renouvellement période d'essai
["renouvellement période d'essai", "renouvellement période d'essai lettre", "motif renouvellement période d'essai", "renouvellement de la période d'essai", "renouvellement de la période d'essai et cdd", "renouvellement période d'essai", "renouvellement période d'essai cdd"]

Disambiguation match : rupture conventionnelle
['rupture conventionnelle', 'rupture conventionnelle cdi']

Disambiguation match : licenciement
['licenciement', 'licenciement']

Disambiguation match : heures supplémentaires
['heures supplémentaires', 'heures supplémentaires']

Disambiguation match : congés payés
['congés payés', 'congés payés']

Disambiguation match : indemnité de licenciement
['indemnité de licenciement', 'indemnité de licenciement']

Disambiguation match : congés pour événements familiaux
['congés pour événements familiaux', 'les congés pour événements familiaux']

Disambiguation match : convention collective
['convention collective', 'conventi

Disambiguation match : rôle convention collective
['convention collective', 'convention collective', 'rôle convention collective']

Disambiguation match : comment mettre fin à un cdi
['comment mettre fin à un cdd?', 'comment mettre fin à un cdi ?', 'comment mettre fin à un cdi']

Disambiguation match : jours de fractionnement
['jours de fractionnement', 'jours de fractionnement syntec', 'jours de fractionnement', 'calcul jours de fractionnement']

Disambiguation match : congé paternité
['congé paternité', 'congé paternité']

Disambiguation match : arrêt maladie et préavis
['arrêt maladie et préavis', 'arrêt maladie et préavis']

Disambiguation match : quelle est la durée de ma période d'essaie
["quelle est la durée de la période d'essai ?", 'quelle est la durée maximale de la période d’essai ?', "quelle est la durée de ma période d'essaie"]

Disambiguation match : comité social et économique
['comité social et économique', 'comité social et économique']

Disambiguation match : les cong

Disambiguation match : congé de maternité
['congé de maternité', 'congé de maternité']

Disambiguation match : congé maternité et accouchement prématuré
['congé maternité et accouchement prématuré', 'congé maternité et accouchement tardif']

Disambiguation match : congé maternité et accouchement tardif
['congé maternité et accouchement prématuré', 'congé maternité et accouchement tardif']

Disambiguation match : congé parental
['congé parental', 'congé parental']

Disambiguation match : congé paternité
['congé paternité', 'congé paternité']

Disambiguation match : congé sabbatique
['congé sabbatique', 'congé sabbatique']

Disambiguation match : congé sans solde
['congé sans solde', 'congés sans solde']

Disambiguation match : congés imposés par l' employeur
["congés imposés par l' employeur", "congés imposés par l' employeur"]

Disambiguation match : congés payés
['congés payés', 'congés payés']

Disambiguation match : congés payés et congé fractionnement
['congés payés et congé fracti

Disambiguation match : fin de période d'essai à l'initiative du salarié
["fin de période d'essai à l'initiative du salarié", "fin de période d'essai à l'initiative du salarié"]

Disambiguation match : fin de période d'essai à l'initiative du salarié
["fin de période d'essai à l'initiative du salarié", "fin de période d'essai à l'initiative du salarié"]

Disambiguation match : formation salarié cif
['formation salarié cif', 'formation salarié cpf', 'formation salarié dif']

Disambiguation match : formation salarié cpf
['formation salarié cif', 'formation salarié cpf', 'formation salarié dif']

Disambiguation match : formation salarié dif
['formation salarié cif', 'formation salarié cpf', 'formation salarié dif']

Disambiguation match : grève
['grève', 'grève']

Disambiguation match : harcèlement moral
['harcèlement moral', 'harcèlement moral']

Disambiguation match : harcèlement moral au travail
['harcèlement moral au travail', 'harcèlement moral au travail']

Disambiguation match : har

Disambiguation match : licenciement pendant un arrêt maladie
['licenciement pendant un arrêt maladie', 'licenciement pendant arrêt maladie']

Disambiguation match : licenciement pour abandon de poste
['licenciement pour abandon de poste', 'licenciement pour abandon de poste']

Disambiguation match : licenciement pour faute
['licenciement pour faute', 'licenciement pour faute']

Disambiguation match : licenciement pour faute grave
['licenciement pour faute grave are', 'licenciement pour faute grave', 'licenciement pour faute grave', 'licenciement pour faute grave']

Disambiguation match : licenciement pour faute grave are
['licenciement pour faute grave are', 'licenciement pour faute grave', 'licenciement pour faute grave', 'licenciement pour faute grave']

Disambiguation match : licenciement pour faute grave indemnité
['indemnité licenciement pour faute grave', 'licenciement pour faute grave indemnité']

Disambiguation match : licenciement pour faute pendant période d'essai
["licenciem

Disambiguation match : préavis cdd
['préavis cdd', 'préavis cdd']

Disambiguation match : préavis de départ à la retraite
['préavis départ à la retraite', 'durée de préavis de départ à la retraite', 'préavis de départ à la retraite']

Disambiguation match : préavis de licenciement économique et congés payés
['préavis de licenciement économique et congés payés', 'préavis de licenciement économique et congés payés']

Disambiguation match : préavis de licenciement économique et congés payés
['préavis de licenciement économique et congés payés', 'préavis de licenciement économique et congés payés']

Disambiguation match : préavis démission
['préavis démission', 'préavis démission']

Disambiguation match : préavis démission congés payés imposés
['préavis démission congés payés imposés', 'préavis démission congés payés imposés']

Disambiguation match : préavis démission congés payés imposés
['préavis démission congés payés imposés', 'préavis démission congés payés imposés']

Disambiguation m

Disambiguation match : refus rupture conventionnelle individuelle
['refus rupture conventionnelle individuelle', 'rupture conventionnelle individuelle']

Disambiguation match : rémunération
['rémunération', 'rémunération']

Disambiguation match : renouvellement de la période d'essai
["renouvellement période d'essai", "motif renouvellement période d'essai", "renouvellement de la période d'essai", "renouvellement de la période d'essai et cdd", "renouvellement période d'essai"]

Disambiguation match : renouvellement de la période d'essai et cdd
["renouvellement période d'essai", "motif renouvellement période d'essai", "renouvellement de la période d'essai", "renouvellement de la période d'essai et cdd", "renouvellement période d'essai"]

Disambiguation match : renouvellement période d'essai
["renouvellement période d'essai", "renouvellement période d'essai lettre", "motif renouvellement période d'essai", "renouvellement de la période d'essai", "renouvellement de la période d'essai et cdd"

Disambiguation match : travail de nuit
['travail de nuit', 'travail de nuit']

Disambiguation match : travail dimanche
['travail dimanche', 'travail dimanche cadre']

Disambiguation match : travail dimanche cadre
['travail dimanche', 'travail dimanche cadre']

Disambiguation match : travail en hauteur
['travail en hauteur', 'travail en hauteur']

Disambiguation match : travail en soirée
['travail en soirée', 'travail en soirée']

Disambiguation match : travail le dimanche
['travail le dimanche', 'travail le dimanche']

Disambiguation match : un employeur peut-il refuser d'accorder un congé sabbatique
['mon employeur peut-il refuser un congé sabbatique ?', "un employeur peut-il refuser d'accorder un congé sabbatique"]

Disambiguation match : versement du salaire
['versement salaire', 'versement du salaire']

Disambiguation match : versement du salaire date
['versement salaire date', 'versement du salaire date']

Disambiguation match : versement salaire
['versement salaire', 'versement d

/cdtn/ubuntu-env/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
disambiguated.shape

(2781, 2)

In [32]:
with open('new_suggs.json', 'w') as f:
    for item in disambiguated.iterrows():
        s = item[1].suggestion
        w = item[1].weight
        j = json.dumps({'entity':s, 'value':w})
        f.write("%s\n" % j)